In [1]:
import numpy as np

import sys
sys.path.insert(
    1,
    '/home/dtd/Documents/interpretable_machine_learning/Source Code/my_work/lib'
)

import data_load
import numpy as np
import pandas as pd
import logging
import incremental_ps_score_estimator as ipse
import utils


from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from geneticalgorithm import geneticalgorithm as ga


In [2]:
file_path = "https://msalicedatapublic.blob.core.windows.net/datasets/Pricing/pricing_sample.csv"
train_data = pd.read_csv(file_path)

In [3]:
train_data['treatment'] = np.where(train_data['price'] == 1, 1, 0)
train_data['price'] = np.where(train_data['price'] == 1, 1, 0.85)

In [4]:
outcome = "demand"
treatment = "treatment"
col = list(train_data.columns)
col.remove("price")
print(col)

cov = col[:]
cov.remove(treatment)
cov.remove(outcome)
cov.remove('income')
print(cov)

features = col[:]
features.remove(outcome)

print(features)

['account_age', 'age', 'avg_hours', 'days_visited', 'friends_count', 'has_membership', 'is_US', 'songs_purchased', 'income', 'demand', 'treatment']
['account_age', 'age', 'avg_hours', 'days_visited', 'friends_count', 'has_membership', 'is_US', 'songs_purchased']
['account_age', 'age', 'avg_hours', 'days_visited', 'friends_count', 'has_membership', 'is_US', 'songs_purchased', 'income', 'treatment']


In [5]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(train_data, test_size=0.7)

In [6]:
## Fit treatment
model_t = LogisticRegression()
model_t.fit(train_data[cov], train_data[treatment])

train_data['p1'] = model_t.predict_proba(train_data[cov])[:,1]
train_data['p0'] = 1 - train_data['p1']

train_data["prediction"] = np.where(train_data["p1"] >= 0.5, 1, 0)

## Fit outcome
model_y = GradientBoostingRegressor(random_state=0, n_estimators = 1000)
model_y.fit(train_data[features], train_data[outcome])

    

GradientBoostingRegressor(n_estimators=1000, random_state=0)

In [7]:
train_data.shape

(3000, 15)

## Optimization

In [48]:
def gamma_fn(X):
    return -3 - 14 * (X["income"] < 1)

def beta_fn(X):
    return 20 + 0.5 * (X["avg_hours"]) + 5 * (X["days_visited"] > 4)

def demand_fn(data, T):
    Y = gamma_fn(data) * T + beta_fn(data)
    return Y

def true_te(x, n, stats):
    if x < 1:
        subdata = df[df["income"] < 1].sample(n=n, replace=True)
    else:
        subdata = df[df["income"] >= 1].sample(n=n, replace=True)
    te_array = subdata["price"] * gamma_fn(subdata) / (subdata["demand"])
    if stats == "mean":
        return np.mean(te_array)
    elif stats == "median":
        return np.median(te_array)
    elif isinstance(stats, int):
        return np.percentile(te_array, stats)
    
def revenue_fn(data, discount_level1, discount_level2, baseline_T, policy):
    policy_price = baseline_T * (1 - discount_level1) * policy + baseline_T * (1 - discount_level2) * (1 - policy)
    demand = demand_fn(data, policy_price)
    rev = demand * policy_price
    return rev

def fitness_function(propensity_score, solution_idx):
    policy = np.where(propensity_score >= 0.5, 1, 0)
    return np.mean(revenue_fn(train_data,  0., 0.15, 1, policy))


In [49]:
train_data.head()

,account_age,age,avg_hours,days_visited,friends_count,has_membership,is_US,songs_purchased,income,price,demand,treatment,p1,p0,prediction
3784,2,60,5.852842,2,9,1,1,10.309400,0.807694,0.85,9.326421,0,0.405256,0.594744,0
3491,4,53,2.461606,6,9,0,1,7.323661,1.289689,1.00,23.230803,1,0.440003,0.559997,0
6323,1,43,6.708785,3,7,1,1,6.139332,1.211041,1.00,20.354392,1,0.420152,0.579848,0
1899,5,31,6.188160,6,6,1,1,6.822765,1.983065,1.00,25.094080,1,0.417860,0.582140,0
8836,5,36,6.161451,7,12,0,1,5.190748,0.390405,0.85,12.780725,0,0.421466,0.578534,0


In [50]:
#0: discount, level 2, 1: no discount - level 1
np.mean(revenue_fn(train_data,  0.0, 0.15, 1, train_data.treatment.values))

14.20314236151327

In [51]:
np.mean(train_data['demand']*train_data['price'])

14.274315694846607

## PyGrad

In [53]:
import pygad
import numpy

function_inputs = np.random.rand(train_data.shape[0]) # Function inputs.

num_generations = 5000 # Number of generations.
num_parents_mating = 500 # Number of solutions to be selected as parents in the mating pool.

# To prepare the initial population, there are 2 ways:
# 1) Prepare it yourself and pass it to the initial_population parameter. This way is useful when the user wants to start the genetic algorithm with a custom initial population.
# 2) Assign valid integer values to the sol_per_pop and num_genes parameters. If the initial_population parameter exists, then the sol_per_pop and num_genes parameters are useless.
sol_per_pop = 1500 # Number of solutions in the population.
num_genes = len(function_inputs)

init_range_low = -1
init_range_high = 1

parent_selection_type = "sss" # Type of parent selection.
keep_parents = -1 # Number of parents to keep in the next population. -1 means keep all parents and 0 means keep nothing.

crossover_type = "two_points" # Type of the crossover operator.

# Parameters of the mutation operation.
mutation_type = "random" # Type of the mutation operator.
mutation_percent_genes = 20 # Percentage of genes to mutate. This parameter has no action if the parameter mutation_num_genes exists or when mutation_type is None.

last_fitness = 0
def callback_generation(ga_instance):
    global last_fitness
    print("Generation = {generation}".format(generation=ga_instance.generations_completed))
    print("Fitness    = {fitness}".format(fitness=ga_instance.best_solution()[1]))
    print("Change     = {change}".format(change=ga_instance.best_solution()[1] - last_fitness))
    print("\n")
    last_fitness = ga_instance.best_solution()[1]

print("Running model")
# Creating an instance of the GA class inside the ga module. Some parameters are initialized within the constructor.
ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating, 
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop, 
                       num_genes=num_genes,
                       init_range_low=init_range_low,
                       init_range_high=init_range_high,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes,
                       callback_generation=callback_generation)

# Running the GA to optimize the parameters of the function.
ga_instance.run()


Running model
Generation = 1
Fitness    = 13.635103254783862
Change     = 13.635103254783862


Generation = 2
Fitness    = 13.648487527175904
Change     = 0.013384272392041652


Generation = 3
Fitness    = 13.66614248705344
Change     = 0.01765495987753596


Generation = 4
Fitness    = 13.677196655487394
Change     = 0.011054168433954459


Generation = 5
Fitness    = 13.701605680736362
Change     = 0.02440902524896771


Generation = 6
Fitness    = 13.712519959563709
Change     = 0.010914278827346635


Generation = 7
Fitness    = 13.722410302218725
Change     = 0.009890342655015871


Generation = 8
Fitness    = 13.73690526258183
Change     = 0.014494960363105491


Generation = 9
Fitness    = 13.743730739813987
Change     = 0.0068254772321569845


Generation = 10
Fitness    = 13.755241579135868
Change     = 0.011510839321880795


Generation = 11
Fitness    = 13.764632074896591
Change     = 0.009390495760722928


Generation = 12
Fitness    = 13.769652380943734
Change     = 0.0050203060471

Change     = 0.005796918577374299


Generation = 104
Fitness    = 14.165017271604777
Change     = 0.0


Generation = 105
Fitness    = 14.165017271604777
Change     = 0.0


Generation = 106
Fitness    = 14.165017271604777
Change     = 0.0


Generation = 107
Fitness    = 14.1705493280584
Change     = 0.005532056453622758


Generation = 108
Fitness    = 14.1705493280584
Change     = 0.0


Generation = 109
Fitness    = 14.1705493280584
Change     = 0.0


Generation = 110
Fitness    = 14.175376891193606
Change     = 0.004827563135206603


Generation = 111
Fitness    = 14.175376891193606
Change     = 0.0


Generation = 112
Fitness    = 14.175376891193606
Change     = 0.0


Generation = 113
Fitness    = 14.177144873169894
Change     = 0.0017679819762879134


Generation = 114
Fitness    = 14.177144873169894
Change     = 0.0


Generation = 115
Fitness    = 14.186955741766553
Change     = 0.009810868596659361


Generation = 116
Fitness    = 14.186955741766553
Change     = 0.0


Generation = 117


Generation = 211
Fitness    = 14.309418330427611
Change     = 0.0


Generation = 212
Fitness    = 14.309418330427611
Change     = 0.0


Generation = 213
Fitness    = 14.316254174063948
Change     = 0.006835843636336492


Generation = 214
Fitness    = 14.316254174063948
Change     = 0.0


Generation = 215
Fitness    = 14.316254174063948
Change     = 0.0


Generation = 216
Fitness    = 14.316254174063948
Change     = 0.0


Generation = 217
Fitness    = 14.316254174063948
Change     = 0.0


Generation = 218
Fitness    = 14.316254174063948
Change     = 0.0


Generation = 219
Fitness    = 14.31854759656032
Change     = 0.0022934224963719885


Generation = 220
Fitness    = 14.321227145842247
Change     = 0.0026795492819271516


Generation = 221
Fitness    = 14.324470867574695
Change     = 0.0032437217324474688


Generation = 222
Fitness    = 14.324470867574695
Change     = 0.0


Generation = 223
Fitness    = 14.324470867574695
Change     = 0.0


Generation = 224
Fitness    = 14.3244708675746

Change     = 0.0


Generation = 322
Fitness    = 14.382125556550468
Change     = 0.0


Generation = 323
Fitness    = 14.382125556550468
Change     = 0.0


Generation = 324
Fitness    = 14.382125556550468
Change     = 0.0


Generation = 325
Fitness    = 14.382125556550468
Change     = 0.0


Generation = 326
Fitness    = 14.382125556550468
Change     = 0.0


Generation = 327
Fitness    = 14.382125556550468
Change     = 0.0


Generation = 328
Fitness    = 14.382125556550468
Change     = 0.0


Generation = 329
Fitness    = 14.384825093720751
Change     = 0.0026995371702831505


Generation = 330
Fitness    = 14.384825093720751
Change     = 0.0


Generation = 331
Fitness    = 14.384825093720751
Change     = 0.0


Generation = 332
Fitness    = 14.384825093720751
Change     = 0.0


Generation = 333
Fitness    = 14.39145197700138
Change     = 0.006626883280627993


Generation = 334
Fitness    = 14.39145197700138
Change     = 0.0


Generation = 335
Fitness    = 14.39145197700138
Change     = 0.0

Generation = 439
Fitness    = 14.420440615310584
Change     = 0.0002822448697017421


Generation = 440
Fitness    = 14.420440615310584
Change     = 0.0


Generation = 441
Fitness    = 14.420440615310584
Change     = 0.0


Generation = 442
Fitness    = 14.420440615310584
Change     = 0.0


Generation = 443
Fitness    = 14.420440615310584
Change     = 0.0


Generation = 444
Fitness    = 14.42116149268404
Change     = 0.0007208773734568297


Generation = 445
Fitness    = 14.422090594259629
Change     = 0.0009291015755881205


Generation = 446
Fitness    = 14.422090594259629
Change     = 0.0


Generation = 447
Fitness    = 14.422090594259629
Change     = 0.0


Generation = 448
Fitness    = 14.422090594259629
Change     = 0.0


Generation = 449
Fitness    = 14.422090594259629
Change     = 0.0


Generation = 450
Fitness    = 14.42256036341696
Change     = 0.00046976915733054625


Generation = 451
Fitness    = 14.42372094948367
Change     = 0.001160586066710323


Generation = 452
Fitness    =

...

Fitness    = 14.53645575480562
Change     = 0.0


Generation = 1496
Fitness    = 14.53645575480562
Change     = 0.0


Generation = 1497
Fitness    = 14.53645575480562
Change     = 0.0


Generation = 1498
Fitness    = 14.53645575480562
Change     = 0.0


Generation = 1499
Fitness    = 14.53645575480562
Change     = 0.0


Generation = 1500
Fitness    = 14.53645575480562
Change     = 0.0


Generation = 1501
Fitness    = 14.53645575480562
Change     = 0.0


Generation = 1502
Fitness    = 14.53645575480562
Change     = 0.0


Generation = 1503
Fitness    = 14.53645575480562
Change     = 0.0


Generation = 1504
Fitness    = 14.53645575480562
Change     = 0.0


Generation = 1505
Fitness    = 14.53645575480562
Change     = 0.0


Generation = 1506
Fitness    = 14.53645575480562
Change     = 0.0


Generation = 1507
Fitness    = 14.53645575480562
Change     = 0.0


Generation = 1508
Fitness    = 14.53645575480562
Change     = 0.0


Generation = 1509
Fitness    = 14.53645575480562
Change     = 0.0


Fitness    = 14.537766640809554
Change     = 0.0


Generation = 1616
Fitness    = 14.537766640809554
Change     = 0.0


Generation = 1617
Fitness    = 14.537766640809554
Change     = 0.0


Generation = 1618
Fitness    = 14.537766640809554
Change     = 0.0


Generation = 1619
Fitness    = 14.537766640809554
Change     = 0.0


Generation = 1620
Fitness    = 14.537766640809554
Change     = 0.0


Generation = 1621
Fitness    = 14.537809395878575
Change     = 4.275506902118309e-05


Generation = 1622
Fitness    = 14.537809395878575
Change     = 0.0


Generation = 1623
Fitness    = 14.537809395878575
Change     = 0.0


Generation = 1624
Fitness    = 14.537809395878575
Change     = 0.0


Generation = 1625
Fitness    = 14.537809395878575
Change     = 0.0


Generation = 1626
Fitness    = 14.537809395878575
Change     = 0.0


Generation = 1627
Fitness    = 14.537809395878575
Change     = 0.0


Generation = 1628
Fitness    = 14.537809395878575
Change     = 0.0


Generation = 1629
Fitness    = 14.

Change     = 0.0


Generation = 1734
Fitness    = 14.541239395051404
Change     = 0.0


Generation = 1735
Fitness    = 14.541239395051404
Change     = 0.0


Generation = 1736
Fitness    = 14.541239395051404
Change     = 0.0


Generation = 1737
Fitness    = 14.541239395051404
Change     = 0.0


Generation = 1738
Fitness    = 14.541239395051404
Change     = 0.0


Generation = 1739
Fitness    = 14.541239395051404
Change     = 0.0


Generation = 1740
Fitness    = 14.541239395051404
Change     = 0.0


Generation = 1741
Fitness    = 14.541239395051404
Change     = 0.0


Generation = 1742
Fitness    = 14.541239395051404
Change     = 0.0


Generation = 1743
Fitness    = 14.541239395051404
Change     = 0.0


Generation = 1744
Fitness    = 14.54131935723185
Change     = 7.996218044681314e-05


Generation = 1745
Fitness    = 14.54131935723185
Change     = 0.0


Generation = 1746
Fitness    = 14.54131935723185
Change     = 0.0


Generation = 1747
Fitness    = 14.54131935723185
Change     = 0.0


G

Change     = 0.0


Generation = 1851
Fitness    = 14.54820064489849
Change     = 0.0


Generation = 1852
Fitness    = 14.54820064489849
Change     = 0.0


Generation = 1853
Fitness    = 14.54820064489849
Change     = 0.0


Generation = 1854
Fitness    = 14.54820064489849
Change     = 0.0


Generation = 1855
Fitness    = 14.54820064489849
Change     = 0.0


Generation = 1856
Fitness    = 14.54820064489849
Change     = 0.0


Generation = 1857
Fitness    = 14.54820064489849
Change     = 0.0


Generation = 1858
Fitness    = 14.54820064489849
Change     = 0.0


Generation = 1859
Fitness    = 14.54820064489849
Change     = 0.0


Generation = 1860
Fitness    = 14.54820064489849
Change     = 0.0


Generation = 1861
Fitness    = 14.54820064489849
Change     = 0.0


Generation = 1862
Fitness    = 14.54820064489849
Change     = 0.0


Generation = 1863
Fitness    = 14.54820064489849
Change     = 0.0


Generation = 1864
Fitness    = 14.54820064489849
Change     = 0.0


Generation = 1865
Fitness    

Fitness    = 14.54820064489849
Change     = 0.0


Generation = 1972
Fitness    = 14.54820064489849
Change     = 0.0


Generation = 1973
Fitness    = 14.54820064489849
Change     = 0.0


Generation = 1974
Fitness    = 14.54820064489849
Change     = 0.0


Generation = 1975
Fitness    = 14.54820064489849
Change     = 0.0


Generation = 1976
Fitness    = 14.54820064489849
Change     = 0.0


Generation = 1977
Fitness    = 14.54820064489849
Change     = 0.0


Generation = 1978
Fitness    = 14.54820064489849
Change     = 0.0


Generation = 1979
Fitness    = 14.54820064489849
Change     = 0.0


Generation = 1980
Fitness    = 14.54820064489849
Change     = 0.0


Generation = 1981
Fitness    = 14.54820064489849
Change     = 0.0


Generation = 1982
Fitness    = 14.54820064489849
Change     = 0.0


Generation = 1983
Fitness    = 14.54820064489849
Change     = 0.0


Generation = 1984
Fitness    = 14.54820064489849
Change     = 0.0


Generation = 1985
Fitness    = 14.54820064489849
Change     = 0.0


In [ ]:
-----------------------------------------

In [55]:
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Parameters of the best solution : {solution}".format(solution=solution))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))

Parameters of the best solution : [-0.71313312  2.48229279  3.62089145 ...  3.87448791  5.50473923
  6.35494275]
Fitness value of the best solution = 14.548779030793467


In [56]:
proposed_policy = np.where(solution >= 0.5, 1, 0)

In [57]:
proposed_policy

array([0, 1, 1, ..., 1, 1, 1])

In [60]:
discount_all = np.mean(revenue_fn(train_data,  0.0, 0.15, 1, np.zeros(train_data.shape[0])))
no_discount = np.mean(revenue_fn(train_data,  0.0, 0.15, 1, np.ones(train_data.shape[0])))

In [62]:
print("Current policy {:.4f}".format(discount_all))
print("Discount all {:.4f}".format(discount_all))
print("No discount {:.4f}".format(no_discount))
print("-"*100)
print("Proposed strategy {:.4f}".format(solution_fitness))

Current policy 13.3610
Discount all 13.3610
No discount 14.1852
----------------------------------------------------------------------------------------------------
Proposed strategy 14.5488


In [65]:
from collections import Counter 
print(Counter(proposed_policy))
print(Counter(train_data.treatment.values))


Counter({1: 1651, 0: 1349})
Counter({0: 1712, 1: 1288})


In [67]:
current_policy = train_data.treatment.values

In [71]:
proposed_policy = [str(x) for x in proposed_policy]
current_policy = [str(x) for x in current_policy]

In [74]:
14.5488 / 13.3610 * 100

108.89005313973506